In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 4: Tree Building assignment
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [122]:
import pandas as pd
import os

week = 3  # from last module

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table = pd.read_csv(home_path + file_path + file_name)

In [123]:
os.chdir(home_path + file_path + 'datascience_1/')
!git pull

Already up-to-date.


In [124]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + '.../datascience_1')
from week4 import *
%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 f1	 find_best_splitter	 generate_table	 gig	 gini	 
informedness	 path_id	 predictor_case	 probabilities	 reorder_paths	 tree_predictor	 


In [125]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,pred_1,pred_1_type,pred_2,pred_2_type,apin_binned,apin_Low,apin_Average,apin_High,apin_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,1,true_positive,1,true_positive,Low,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,1,false_positive,1,false_positive,Low,1,0,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,1,true_positive,1,true_positive,Low,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,1,true_positive,1,true_positive,Low,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,1,true_positive,1,true_positive,Low,1,0,0,0


In [126]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'bin_lam', 'bin_Low', 'bin_Average',
       'bin_High', 'ch_bad', 'ch_good', 'ch_nan', 'pred_1', 'pred_1_type',
       'pred_2', 'pred_2_type', 'apin_binned', 'apin_Low', 'apin_Average',
       'apin_High', 'apin_nan'], dtype=object)

<hr>
<h1>
1: Wrangle
</h1>
<p>
<div class=h1_cell>
<p>
Drop these columns: 'pred_1', 'pred_1_type', 'pred_2', 'pred_2_type'
<p>
Produce the following columns in standard way: 'dep_0', 'dep_1', 'dep_2', 'dep_3+'
</div>

In [127]:
loan_table = loan_table.drop(['pred_1', 'pred_1_type','pred_2', 'pred_2_type'], axis=1)

In [128]:
one_hot_emb = pd.get_dummies(loan_table['Dependents'],prefix='dep',dummy_na=False)
loan_table = loan_table.join(one_hot_emb)


In [129]:
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0


<hr>
<h1>
2: Find the best depth
</h1>
<p>
<div class=h1_cell>
<p>
Explore depths from 1 to 5. Build a tree for each depth and test it. Plot the results.
<p>
First, define the columns to use. I do that for you below.
</div>

In [130]:
column_candidates = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_Low', 'apin_High', 'apin_Average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'bin_Low', 'bin_Average', 'bin_High',
        #Credit_History
        'ch_bad', 'ch_good']

In [131]:
#print(column_candidates[0:3])
tree1 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':1})
tree1['paths']

[{'conjunction': [('ch_bad_1', <function week4.<lambda>>)],
  'gig_score': 0.12559965113188631,
  'prediction': 0},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>)],
  'gig_score': 0.12559965113188631,
  'prediction': 1}]

<div class=h1_cell>
<p>
As reminder, a tree with only one splitter node is called a stump. You can see that `ch_bad` is the best splitter with only one node. Thought question: As we build deeper trees below, is it possible for the root node to be something other than `ch_bad`?
</div>

In [132]:
tree2 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':2})
print(len(tree2['paths']))
tree2['paths']

4


[{'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_1', <function week4.<lambda>>)],
  'gig_score': 0.019292789018833467,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>)],
  'gig_score': 0.019292789018833467,
  'prediction': 0},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_1', <function week4.<lambda>>)],
  'gig_score': 0.006462619030620731,
  'prediction': 1},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_0', <function week4.<lambda>>)],
  'gig_score': 0.006462619030620731,
  'prediction': 1}]

In [133]:
tree3 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':3})
print(len(tree3['paths']))
tree3['paths']

7


[{'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_1', <function week4.<lambda>>)],
  'gig_score': 0.019292789018833467,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>),
   ('pa_Urban_1', <function week4.<lambda>>)],
  'gig_score': 0.0014037192421204325,
  'prediction': 0},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>),
   ('pa_Urban_0', <function week4.<lambda>>)],
  'gig_score': 0.0014037192421204325,
  'prediction': 0},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_1', <function week4.<lambda>>),
   ('bin_Average_1', <function week4.<lambda>>)],
  'gig_score': 0.0052148575352743975,
  'prediction': 1},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_1', <function week4.<lambda>>),
   ('bin_Average_0', <function week4.<lambda>>)],
  'gig_score': 0.005214857535

In [134]:
tree4 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':4})
print(len(tree4['paths']))
#tree4['paths']

13


In [135]:
tree5 = build_tree_iter(loan_table, column_candidates, 'Loan_Status', {'max-depth':5})
print(len(tree5['paths']))
#tree5['paths']

20


<div class=h1_cell>
<p>
Show your exploration work below.
</div>

In [136]:
#add results to loan_table for tree1
loan_table['tree_1'] = loan_table.apply(lambda row: tree_predictor(row, tree1), axis=1)
loan_table['tree_1_type'] = loan_table.apply(lambda row: predictor_case(row, pred='tree_1', target='Loan_Status'), axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive


In [137]:
#add results to loan_table for tree2
loan_table['tree_2'] = loan_table.apply(lambda row: tree_predictor(row, tree2), axis=1)
loan_table['tree_2_type'] = loan_table.apply(lambda row: predictor_case(row, pred='tree_2', target='Loan_Status'), axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type,tree_2,tree_2_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive


In [138]:
#add results to loan_table for tree3
loan_table['tree_3'] = loan_table.apply(lambda row: tree_predictor(row, tree3), axis=1)
loan_table['tree_3_type'] = loan_table.apply(lambda row: predictor_case(row, pred='tree_3', target='Loan_Status'), axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type,tree_2,tree_2_type,tree_3,tree_3_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive


In [139]:
#add results to loan_table for tree4
loan_table['tree_4'] = loan_table.apply(lambda row: tree_predictor(row, tree4), axis=1)
loan_table['tree_4_type'] = loan_table.apply(lambda row: predictor_case(row, pred='tree_4', target='Loan_Status'), axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type,tree_2,tree_2_type,tree_3,tree_3_type,tree_4,tree_4_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [140]:
#add results to loan_table for tree5
loan_table['tree_5'] = loan_table.apply(lambda row: tree_predictor(row, tree5), axis=1)
loan_table['tree_5_type'] = loan_table.apply(lambda row: predictor_case(row, pred='tree_5', target='Loan_Status'), axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type,tree_2,tree_2_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [141]:
tree_1_types = loan_table['tree_1_type'].value_counts()
tree_2_types = loan_table['tree_2_type'].value_counts()
tree_3_types = loan_table['tree_3_type'].value_counts()
tree_4_types = loan_table['tree_4_type'].value_counts()
tree_5_types = loan_table['tree_5_type'].value_counts()

In [142]:
print((accuracy(tree_1_types), f1(tree_1_types), informedness(tree_1_types)))

(0.8094462540716613, 0.8764519535374868, 0.4104956556082149)


In [143]:
print((accuracy(tree_2_types), f1(tree_2_types), informedness(tree_2_types)))

(0.8110749185667753, 0.8776371308016878, 0.41286532385466024)


In [144]:
print((accuracy(tree_3_types), f1(tree_3_types), informedness(tree_3_types)))

(0.8110749185667753, 0.8776371308016878, 0.41286532385466024)


In [145]:
print((accuracy(tree_4_types), f1(tree_4_types), informedness(tree_4_types)))

(0.8127035830618893, 0.8785638859556495, 0.4180736571879937)


In [146]:
print((accuracy(tree_5_types), f1(tree_5_types), informedness(tree_5_types)))

(0.8192182410423453, 0.8802588996763754, 0.46161631121642976)


<div class=h1_cell>
<p>
Should we stop at 5? It looks like our measures are continuing to go up. I'll let you stop here but in practice it would be good to find where depth is optimal.
</div>

<hr>
<h1>
3: Order the paths in a tree according to most used
</h1>
<p>
<div class=h1_cell>
<p>
One of the cool things about building our own code for doing predictions with decision trees is that we can explore various options directly. Let's do that now. Look at this code in `tree_predictor`:
<p>
```
for path in tree['paths']:
```
<p>
It is doing a sequential and ordered enumeration of the paths in a tree. We can gain some efficiency by ordering the paths, right? If we can determine which paths are most likely to succeed, we can try those paths first. In practice, this means reordeing the list of dictionaries in tree['paths'] so most used path is first, and least used path is last. I'd like you to write some code that will reorder `tree['paths']` on this basis of most used.
<p>
I'll break it up into pieces. First, define a function `path_id` that accepts a row and a tree and returns the index of the path that succeeded. So if the first path in tree['paths'] succeeded, you would return its index 0.
</div>

In [147]:
def path_id(row, tree):    
    #go through each path, one by one (could use a map instead of for loop?)
    for index, path in enumerate(tree['paths']):
        conjuncts = path['conjunction']
        result = map(lambda(tuple): tuple[1](row), conjuncts)  # potential to be parallelized
        if all(result):
            return index
    raise LookupError('No true paths found for row: ' + str(row))



<div class=h1_cell>

Now apply the function against the loan table using tree1 and produce a series called `pcounts1`.
</div>


In [148]:
pcounts1 = loan_table.apply(lambda row: path_id(row, tree1), axis=1)  # tree1 has only 2 paths - it's a stump

pcounts1.value_counts()

1    525
0     89
dtype: int64

<div class=h1_cell>

Same for tree2.
</div>

In [149]:
pcounts2 = loan_table.apply(lambda row: path_id(row, tree2), axis=1)

pcounts2.value_counts()

3    322
2    203
1     88
0      1
dtype: int64

<div class=h1_cell>
Same for tree3.
</div>

In [150]:
pcounts3 = loan_table.apply(lambda row: path_id(row, tree3), axis=1)

pcounts3.value_counts()

6    267
4    187
2     57
5     55
1     31
3     16
0      1
dtype: int64

<div class=h1_cell>

Let's use tree1 (i.e., pcounts1) as our focus. Turn pcounts1.value_counts() into a dictionary.
</div>

In [151]:

pdict1 = dict(pcounts1.value_counts())
pdict1

{0: 89, 1: 525}

<div class=h1_cell>

Now turn the dictionary into a list of tuples.
</div>

In [152]:
plist1 = pdict1.items()
plist1

[(0, 89), (1, 525)]

<div class=h1_cell>
Now sort the list so most used is first.
</div>

In [153]:
plist1 = sorted(plist1, key=lambda x: x[1], reverse=True)
plist1

[(1, 525), (0, 89)]

<div class=h1_cell>
Finally, build a new path list using plist as a guide.
</div>

In [154]:

paths = tree1['paths']
new_paths = []

for item in plist1:
    compareto = item[0].item() # a numpy int
    #print(path)
    for index, path in enumerate(paths):
        if index == compareto:
            new_paths.append(path)
            

for path2 in newpaths: 
    print("NEWLIST---")
    print(path2)

# keydict = zip(plist1, paths)
# print(paths)
# print(plist1)
# sorted_paths = sorted(keydict, reverse=True)

# print(sorted_paths)
# new_paths = [path for paths, path in sorted_paths]
# print(new_paths)

#paths.sort(key=keydict.get)

#new_paths = []
#paths = tree1['paths']




#for path in paths: 
#    print(path.get('prediction'))

In [155]:
new_paths




[{'conjunction': [('ch_bad_0', <function week4.<lambda>>)],
  'gig_score': 0.12559965113188631,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>)],
  'gig_score': 0.12559965113188631,
  'prediction': 0}]

<hr>
<h2>
Put it all together in a function
</h2>
<p>
<div class=h1_cell>
<p>
I started `reorder_paths` for you below. Please complete it using the pieces you have built above. It should produce a list of paths that is sorted on most frequently used. You can use the output of the function as:
<p>
```

    ordered_paths3 = reorder_paths(loan_table, tree3)
    tree3['paths'] = ordered_paths3
```
<p>
I'll give you some examples below that you can test on - match my output. I added 2 print statements in my function to show you the values I have before sorting and after sorting.
</div>

In [156]:
def reorder_paths(table, tree):
    
    tempcounts = table.apply(lambda row: path_id(row, tree), axis=1)
    tempcounts= tempcounts.value_counts()
    dictcounts = dict(tempcounts)
    listcounts = dictcounts.items()
    sortedcounts = sorted(listcounts, key=lambda x: x[1], reverse=True)
    newlist = []
    
    for item in sortedcounts:
        compareto = item[0].item() # a numpy int
    #print(path)
        for index, path in enumerate(tree['paths']):
            if index == compareto:
                newlist.append(path)
    out = newlist        
    return out

In [157]:
print(reorder_paths(loan_table, tree1))


# tree1['paths'] = reorder_paths(loan_table, tree1)
# print(tree1['paths'])




[{'gig_score': 0.12559965113188631, 'conjunction': [('ch_bad_0', <function <lambda> at 0x7f6b05431ed8>)], 'prediction': 1}, {'gig_score': 0.12559965113188631, 'conjunction': [('ch_bad_1', <function <lambda> at 0x7f6b053b9aa0>)], 'prediction': 0}]


In [158]:
reorder_paths(loan_table, tree2)



[{'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_0', <function week4.<lambda>>)],
  'gig_score': 0.006462619030620731,
  'prediction': 1},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_1', <function week4.<lambda>>)],
  'gig_score': 0.006462619030620731,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>)],
  'gig_score': 0.019292789018833467,
  'prediction': 0},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_1', <function week4.<lambda>>)],
  'gig_score': 0.019292789018833467,
  'prediction': 1}]

In [159]:
reorder_paths(loan_table, tree3)

[{'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_0', <function week4.<lambda>>),
   ('dep_2_0', <function week4.<lambda>>)],
  'gig_score': 0.0021858669417437193,
  'prediction': 1},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_1', <function week4.<lambda>>),
   ('bin_Average_0', <function week4.<lambda>>)],
  'gig_score': 0.0052148575352743975,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>),
   ('pa_Urban_0', <function week4.<lambda>>)],
  'gig_score': 0.0014037192421204325,
  'prediction': 0},
 {'conjunction': [('ch_bad_0', <function week4.<lambda>>),
   ('pa_Semiurban_0', <function week4.<lambda>>),
   ('dep_2_1', <function week4.<lambda>>)],
  'gig_score': 0.0021858669417437193,
  'prediction': 1},
 {'conjunction': [('ch_bad_1', <function week4.<lambda>>),
   ('apin_Average_0', <function week4.<lambda>>),
   ('pa_Urban_1', <function week4.<lam

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Save the table so can use it in next module.
</div>

In [160]:
import os

week = 4  # change this each week

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table.to_csv(home_path + file_path + file_name, index=False)